In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from scipy import stats
from mlxtend.preprocessing import minmax_scaling
import pyttsx3
from pycaret.classification import setup
from pycaret.classification import compare_models
engine = pyttsx3.init()
newVoiceRate = 150
engine.setProperty('rate',newVoiceRate)
vid = "HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_ZIRA_11.0"
engine.setProperty('voice', vid)


In [2]:
df=pd.read_csv("train.csv")

In [3]:
df.drop(['ID'],axis=1,inplace=True)
df.head()

,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead
0,Female,73,RG268,Other,X3,43,No,1045696,No,0
1,Female,30,RG277,Salaried,X1,32,No,581988,No,0
2,Female,56,RG268,Self_Employed,X3,26,No,1484315,Yes,0
3,Male,34,RG270,Salaried,X1,19,No,470454,No,0
4,Female,30,RG282,Salaried,X1,33,No,886787,No,0


In [4]:
df.shape

(245725, 10)

In [5]:
target = str("Is_Lead")
dtypedf = pd.DataFrame()
dtypedf['Feature'] = df.dtypes.index
dtypedf['dtype'] = [str(x) for x in df.dtypes]
numdf = df.loc[:,df.dtypes!=np.object]
catdf = df.loc[:,df.dtypes==np.object]
cat = 0

In [74]:
strout = ""
if df[target].dtypes==np.object:
    cat=1
strn="There are "+str(df.shape[0])+" rows and "+str(df.shape[1])+"columns in the Data. "
strn=strn + " There are xy Numerical features in dataframe. "
strn = strn.replace("xy",str(len(numdf.columns)))
engine.say(strn)
strout = strout +strn
strn="There are xy Categorical features in dataframe. "
strn = strn.replace("xy",str(len(catdf.columns)))
engine.say(strn)

In [75]:
engine.say("Hello")

In [76]:
strout = strout +strn
if(df.isnull().sum().sum()>0):
    strn = "There are "+str(df.isnull().sum().sum())+" Missing values in the data. Which will be Replaced by Mean Mode Imputaion Method."
    engine.say(strn)
    strout = strout +strn
else:
    strn = "There are no Missing values in data"
    engine.say(strn)
    strout = strout +strn

In [77]:
for i in numdf.columns:
    df[i].fillna(df[i].mean(skipna=True),inplace=True)
for i in catdf.columns:
    df[i].fillna(df[i].mode()[0],inplace=True)
for i in catdf.columns:
    le = LabelEncoder()
    le.fit(df[i])
    df[i]= pd.Series(le.transform(df[i]))
for i in numdf.columns:
    df[i]=minmax_scaling(df[i], columns = [0])

In [10]:
x = pd.DataFrame(setup(data = df, target = target, session_id=123,silent=True))

,Description,Value
0,session_id,123
1,Target,Is_Lead
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(245725, 10)"
5,Missing Values,False
6,Numeric Features,9
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [11]:
j=0
for i in range(len(x[0])):
    try:
        if(x[0][i][0][0]=="Setup Config"):
            j=i
    except:
        pass

In [12]:
setting_up = pd.DataFrame(setup(data = df, target = target, session_id=123,silent=True))[0][j][0][1]

,Description,Value
0,session_id,123
1,Target,Is_Lead
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(245725, 10)"
5,Missing Values,False
6,Numeric Features,9
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [13]:
a = compare_models(exclude = ["et"], sort = "AUC",n_select=3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.7935,0.7891,0.2706,0.6619,0.3841,0.2855,0.3268,1.5990
catboost,CatBoost Classifier,0.7920,0.7865,0.2866,0.6407,0.3960,0.2918,0.3264,60.1580
gbc,Gradient Boosting Classifier,0.7912,0.7826,0.2145,0.7006,0.3281,0.2438,0.3043,12.0600
ada,Ada Boost Classifier,0.7811,0.7617,0.1971,0.6274,0.2999,0.2100,0.2599,3.6280
rf,Random Forest Classifier,0.7775,0.7569,0.3161,0.5574,0.4034,0.2793,0.2963,11.3410
qda,Quadratic Discriminant Analysis,0.7398,0.7253,0.2868,0.4298,0.3440,0.1897,0.1958,0.4020
lda,Linear Discriminant Analysis,0.7521,0.7183,0.1395,0.4352,0.2112,0.1082,0.1330,0.4290
lr,Logistic Regression,0.7554,0.7166,0.0966,0.4371,0.1582,0.0788,0.1102,1.8620
nb,Naive Bayes,0.7148,0.7155,0.4455,0.4090,0.4264,0.2372,0.2375,0.1240
knn,K Neighbors Classifier,0.7634,0.7062,0.3366,0.5042,0.4037,0.2633,0.2717,4.2180


In [78]:
test=pd.read_csv("test.csv")
ids=test['ID']
test.drop(['ID'],axis=1,inplace=True)
test.head()

,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active
0,Male,29,RG254,Other,X1,25,Yes,742366,No
1,Male,43,RG268,Other,X2,49,NaN,925537,No
2,Male,31,RG270,Salaried,X1,14,No,215949,No
3,Male,29,RG272,Other,X1,33,No,868070,No
4,Female,29,RG270,Other,X1,19,No,657087,No


In [33]:
from lightgbm import LGBMClassifier

In [43]:
lg= LGBMClassifier(random_state=5)

In [79]:
y=df["Is_Lead"]
df.drop(['Is_Lead'],axis=1,inplace=True)

In [44]:
lg.fit(df,np.array(y))

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=5, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [39]:
y.value_counts()

0.0    187437
1.0     58288
Name: Is_Lead, dtype: int64

In [47]:
df = test
dtypedf = pd.DataFrame()
dtypedf['Feature'] = df.dtypes.index
dtypedf['dtype'] = [str(x) for x in df.dtypes]
numdf = df.loc[:,df.dtypes!=np.object]
catdf = df.loc[:,df.dtypes==np.object]
cat = 0
strout = ""
strn="There are "+str(df.shape[0])+" rows and "+str(df.shape[1])+"columns in the Data. "
strn=strn + " There are xy Numerical features in dataframe. "
strn = strn.replace("xy",str(len(numdf.columns)))
engine.say(strn)
strout = strout +strn
strn="There are xy Categorical features in dataframe. "
strn = strn.replace("xy",str(len(catdf.columns)))
engine.say(strn)
strout = strout +strn
if(df.isnull().sum().sum()>0):
    strn = "There are "+str(df.isnull().sum().sum())+" Missing values in the data. Which will be Replaced by Mean Mode Imputaion Method."
    engine.say(strn)
    strout = strout +strn
else:
    strn = "There are no Missing values in data"
    engine.say(strn)
    strout = strout +strn
for i in numdf.columns:
    df[i].fillna(df[i].mean(skipna=True),inplace=True)
for i in catdf.columns:
    df[i].fillna(df[i].mode()[0],inplace=True)
for i in catdf.columns:
    le = LabelEncoder()
    le.fit(df[i])
    df[i]= pd.Series(le.transform(df[i]))
for i in numdf.columns:
    df[i]=minmax_scaling(df[i], columns = [0])

In [48]:
v=lg.predict_proba(df)

In [56]:
ls=list()

In [57]:
for i in v:
    ls.append(i[1])

In [52]:
lg.classes_

array([0., 1.])

In [54]:
v[0]

array([0.96837908, 0.03162092])

In [63]:
ans=pd.DataFrame()
ans['ID']=ids
ans['Is_Lead']=ls

In [64]:
ans.head()

,ID,Is_Lead
0,VBENBARO,0.031621
1,CCMEWNKY,0.232302
2,VK3KGA9M,0.074052
3,TT8RPZVC,0.034657
4,SHQZEYTZ,0.030068


In [67]:
ans.to_csv("lgbm.csv",index=False)

In [69]:
## sk learn 

from sklearn.ensemble import ExtraTreesClassifier as et

In [80]:
etree = et(n_estimators=100, random_state=0)

In [81]:
etree.fit(df,np.array(y))

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100,
                     n_jobs=None, oob_score=False, random_state=0, verbose=0,
                     warm_start=False)

In [82]:
df = test
dtypedf = pd.DataFrame()
dtypedf['Feature'] = df.dtypes.index
dtypedf['dtype'] = [str(x) for x in df.dtypes]
numdf = df.loc[:,df.dtypes!=np.object]
catdf = df.loc[:,df.dtypes==np.object]
cat = 0
strout = ""
strn="There are "+str(df.shape[0])+" rows and "+str(df.shape[1])+"columns in the Data. "
strn=strn + " There are xy Numerical features in dataframe. "
strn = strn.replace("xy",str(len(numdf.columns)))
engine.say(strn)
strout = strout +strn
strn="There are xy Categorical features in dataframe. "
strn = strn.replace("xy",str(len(catdf.columns)))
engine.say(strn)
strout = strout +strn
if(df.isnull().sum().sum()>0):
    strn = "There are "+str(df.isnull().sum().sum())+" Missing values in the data. Which will be Replaced by Mean Mode Imputaion Method."
    engine.say(strn)
    strout = strout +strn
else:
    strn = "There are no Missing values in data"
    engine.say(strn)
    strout = strout +strn
for i in numdf.columns:
    df[i].fillna(df[i].mean(skipna=True),inplace=True)
for i in catdf.columns:
    df[i].fillna(df[i].mode()[0],inplace=True)
for i in catdf.columns:
    le = LabelEncoder()
    le.fit(df[i])
    df[i]= pd.Series(le.transform(df[i]))
for i in numdf.columns:
    df[i]=minmax_scaling(df[i], columns = [0])

In [83]:
v=etree.predict_proba(df)
ls=list()
for i in v:
    ls.append(i[1])

In [87]:
ans=pd.DataFrame()
ans['ID']=ids
ans['Is_Lead']=ls
ans.to_csv("etree.csv",index=False)